In [1]:
from underthesea import pos_tag
import re
import string
import csv
import pandas as pd
import numpy as np

In [12]:
def prepros():
    dic = {}
            
    for fi in ['train.txt']:
        with open(fi, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            cnt = -1
            for para in lines:
                cnt += 1
                category, para = para.split('\t', 1)
                for s in pos_tag(para):
                    if s[1] in ('A', 'V') or re.search('^N', s[1]):
                        w = re.sub(r'[' + string.punctuation + ']', ' ', s[0]).strip().lower()
                        if w != '':
                            if w not in dic:
                                dic[w] = [0] * len(lines)
                            dic[w][cnt] += 1

    df = pd.DataFrame(dic)
    df.to_csv('featureVectors_5kS.csv')
    
    with open('test.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for k in dic:
            dic[k] = [0] * len(lines)
        cnt = -1
        for para in lines:
            cnt += 1
            for s in pos_tag(para):
                if s[1] in ('A', 'V') or re.search('^N', s[1]):
                    w = re.sub(r'[' + string.punctuation + ']', ' ', s[0]).strip().lower()
                    if w != '':
                        if w not in dic:
                            continue
                        dic[w][cnt] += 1
    df = pd.DataFrame(dic)
    df.to_csv('testVectors.csv')

In [8]:
prepros()

,Unnamed: 0,gần,học sinh,phải,học,điều kiện,tạm bợ,nlđ,trường,tiểu học,...,alec,baldwin,eminem,meirelles,đại náo,hector babenco,giải nam,william hurt,city of god,maids
0,0,3,2,3,5,1,2,1,4,2,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3473,3473,0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3474,3474,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3475,3475,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3476,3476,0,0,1,0,0,0,0,0,0,...,1,3,1,0,0,0,0,0,0,0


In [2]:
%%time
df = pd.read_csv('featureVectors_6kS.csv')
x_train = df.iloc[:, 1:].values
y_train, y_test = [], []
with open('train.txt', 'r', encoding='utf-8') as f:
    for para in f.readlines():
        y_train.append(para.split('\t')[0])
        
df1 = pd.read_csv('testVectors6k.csv')
x_test = df1.iloc[:, 1:].values
with open('testAns.txt', 'r', encoding='utf-8') as f:
    for para in f.readlines():
        y_test.append(para.strip())

Wall time: 11min 14s


In [3]:
with open('scrapy_test/spiders/Output/_trainSet.txt', 'r', encoding='utf-8') as f:
    for para in f.readlines():
        y_train.append(para.split('\t')[0])

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

y_pred = MultinomialNB().fit(x_train, y_train).predict(x_test)
print(accuracy_score(y_true=y_test, y_pred=y_pred) * 100)
'''for i in range(len(y_test)):
    if y_test[i] != y_pred[i]:
        print(i, ' : ', 'ans: ', y_test[i], '  pred: ', y_pred[i])'''

86.6622864651774


"for i in range(len(y_test)):\n    if y_test[i] != y_pred[i]:\n        print(i, ' : ', 'ans: ', y_test[i], '  pred: ', y_pred[i])"

In [9]:
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

params = {
    'max_depth' : np.arange(500, 1500, 50),
    'min_samples_split': np.arange(1, 150, 20),
    'min_samples_leaf': np.arange(1, 150, 20)
}
grid_tree = HalvingGridSearchCV(estimator = tree.DecisionTreeClassifier(), 
                         param_grid = params, scoring = 'accuracy', 
                        cv = 3, verbose = 3, n_jobs=-1)
grid_tree.fit(x_train, y_train)
print(grid_tree.best_estimator_)
y_pred = grid_tree.predict(x_test)
print(accuracy_score(y_true=y_test, y_pred=y_pred) * 100)
#tree.plot_tree(mytree)

n_iterations: 4
n_required_iterations: 7
n_possible_iterations: 4
min_resources_: 60
max_resources_: 4584
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 1280
n_resources: 60
Fitting 3 folds for each of 1280 candidates, totalling 3840 fits
----------
iter: 1
n_candidates: 427
n_resources: 180
Fitting 3 folds for each of 427 candidates, totalling 1281 fits
----------
iter: 2
n_candidates: 143
n_resources: 540
Fitting 3 folds for each of 143 candidates, totalling 429 fits
----------
iter: 3
n_candidates: 48
n_resources: 1620
Fitting 3 folds for each of 48 candidates, totalling 144 fits
DecisionTreeClassifier(max_depth=850, min_samples_split=101)
63.20630749014454
